# Задача 3: Решение СЛАУ методом LU-разложения
---

### Алгоритм (3 этапа)

1. Вычисление LU-разложения: $A = LU$
2. Решение системы $Ly = b$ (прямая подстановка)
3. Решение системы $Ux = y$ (обратная подстановка)

### LU-разложение (из лекции)

Идея: с помощью операций со строками привести матрицу к верхнетреугольному виду $U$.

На каждом шаге $k$ из $j$-й строки ($j > k$) вычитаем $k$-ю строку, умноженную на множитель:


Матрица $L$ — нижнетреугольная с единицами на диагонали и множителями $l_{jk}$ под ней.

**Условие существования:** $A$ имеет LU-разложение $\iff$ $A$ строго регулярная (все ведущие подматрицы невырожденные).

In [26]:
# ============================================================
# ЭТАП 1: LU-разложение
# По лекции: L_{n-1}...L_2 L_1 A = U,  A = LU
# l_jk = a_jk / a_kk,  k < j <= n 
# ============================================================

def lu_decompose(A):
    n = len(A)

    # копия матрицы — будет превращаться в U
    U = [[A[i][j] for j in range(n)] for i in range(n)]

    # L — нижнетреугольная, единицы на диагонали
    L = [[0.0] * n for _ in range(n)]
    for i in range(n):
        L[i][i] = 1.0

    for k in range(n - 1):                      # по столбцам
        if U[k][k] == 0:
            raise ValueError(
                f"Нулевой ведущий элемент u[{k}][{k}] = 0.\n"
                f"LU-разложение без перестановок невозможно."
            )
        for j in range(k + 1, n):                # по строкам ниже k
            L[j][k] = U[j][k] / U[k][k]          # множитель l_jk
            for i in range(k, n):                 # обновляем строку j
                U[j][i] = U[j][i] - L[j][k] * U[k][i]

    return L, U

In [16]:
# ============================================================
# ЭТАП 2: Прямая подстановка  Ly = b
# L — нижнетреугольная с единицами на диагонали
# y_i = b_i - sum_{j=0}^{i-1} l_ij * y_j
# ============================================================

def forward_substitution(L, b):
    n = len(b)
    y = [0.0] * n
    for i in range(n):
        s = 0.0
        for j in range(i):
            s += L[i][j] * y[j]
        y[i] = b[i] - s          # L[i][i] = 1, делить не нужно
    return y

In [18]:
# ============================================================
# ЭТАП 3: Обратная подстановка  Ux = y
# U — верхнетреугольная
# x_i = (y_i - sum_{j=i+1}^{n-1} u_ij * x_j) / u_ii
# ============================================================

def back_substitution(U, y):
    n = len(y)
    x = [0.0] * n
    for i in range(n - 1, -1, -1):
        s = 0.0
        for j in range(i + 1, n):
            s += U[i][j] * x[j]
        x[i] = (y[i] - s) / U[i][i]
    return x

In [19]:
# ============================================================
# Решение СЛАУ:  Ax = b  =>  LUx = b
#   1) A = LU
#   2) Ly = b  ->  y
#   3) Ux = y  ->  x
# ============================================================

def solve_lu(A, b):
    L, U = lu_decompose(A)
    y = forward_substitution(L, b)
    x = back_substitution(U, y)
    return x, L, U, y

---
## Тестирование

Система 4×4:

$$A = \begin{pmatrix} 2 & 1 & 1 & 0 \\ 4 & 3 & 3 & 1 \\ 8 & 7 & 9 & 5 \\ 6 & 7 & 9 & 8 \end{pmatrix}, \quad b = \begin{pmatrix} 1 \\ 1 \\ 1 \\ 1 \end{pmatrix}$$

In [20]:
A = [
    [2,  1,  1,  0],
    [4,  3,  3,  1],
    [8,  7,  9,  5],
    [6,  7,  9,  8],
]
b = [1, 1, 1, 1]

x, L, U, y = solve_lu(A, b)

In [23]:
# ---------- вспомогательные функции для вывода ----------

def print_matrix(name, M):
    print(f"{name} =")
    for row in M:
        print("  [", "  ".join(f"{v:10.4f}" for v in row), "]")
    print()

def print_vector(name, v):
    print(f"{name} = [{', '.join(f'{vi:.1f}' for vi in v)}]\n")

def mat_mul(A, B):
    n = len(A)
    C = [[0.0] * n for _ in range(n)]
    for i in range(n):
        for j in range(n):
            for k in range(n):
                C[i][j] += A[i][k] * B[k][j]
    return C

def mat_vec(A, x):
    n = len(A)
    result = [0.0] * n
    for i in range(n):
        for j in range(n):
            result[i] += A[i][j] * x[j]
    return result

In [24]:
print("="*60)
print("       РЕЗУЛЬТАТЫ LU-РАЗЛОЖЕНИЯ")
print("="*60)

print_matrix("L (нижнетреугольная)", L)
print_matrix("U (верхнетреугольная)", U)

print("-"*60)
print("       РЕШЕНИЕ СИСТЕМЫ")
print("-"*60)

print_vector("y (из Ly=b)", y)
print_vector("x (из Ux=y) — ОТВЕТ", x)

       РЕЗУЛЬТАТЫ LU-РАЗЛОЖЕНИЯ
L (нижнетреугольная) =
  [     1.0000      0.0000      0.0000      0.0000 ]
  [     2.0000      1.0000      0.0000      0.0000 ]
  [     4.0000      3.0000      1.0000      0.0000 ]
  [     3.0000      4.0000      1.0000      1.0000 ]

U (верхнетреугольная) =
  [     2.0000      1.0000      1.0000      0.0000 ]
  [     0.0000      1.0000      1.0000      1.0000 ]
  [     0.0000      0.0000      2.0000      2.0000 ]
  [     0.0000      0.0000      0.0000      2.0000 ]

------------------------------------------------------------
       РЕШЕНИЕ СИСТЕМЫ
------------------------------------------------------------
y (из Ly=b) = [1.0, -1.0, 0.0, 2.0]

x (из Ux=y) — ОТВЕТ = [1.5, -1.0, -1.0, 1.0]



In [10]:
print("="*60)
print("       ПРОВЕРКА")
print("="*60)

# 1) L*U должно равняться A
LU = mat_mul(L, U)
print_matrix("L * U (должно совпасть с A)", LU)

# 2) A*x должно равняться b
Ax = mat_vec(A, x)
print_vector("A*x (должно совпасть с b)", Ax)
print_vector("b (исходный)", b)

# 3) невязка ||Ax - b||
residual = [Ax[i] - b[i] for i in range(len(b))]
norm_r = max(abs(r) for r in residual)
print(f"Невязка ||Ax - b||_inf = {norm_r:.2e}")

       ПРОВЕРКА
L * U (должно совпасть с A) =
  [     2.0000      1.0000      1.0000      0.0000 ]
  [     4.0000      3.0000      3.0000      1.0000 ]
  [     8.0000      7.0000      9.0000      5.0000 ]
  [     6.0000      7.0000      9.0000      8.0000 ]

A*x (должно совпасть с b) = [1.000000, 1.000000, 1.000000, 1.000000]

b (исходный) = [1.000000, 1.000000, 1.000000, 1.000000]

Невязка ||Ax - b||_inf = 0.00e+00


---
## Проверка через numpy

In [ ]:
import numpy as np

A_np = np.array(A, dtype=float)
b_np = np.array(b, dtype=float)

x_np = np.linalg.solve(A_np, b_np)

print("numpy решение:  ", x_np)
print("наше решение:   ", x)
print()

diff = max(abs(x[i] - x_np[i]) for i in range(len(x)))
print(f"макс. расхождение с numpy: {diff:.1e}")

numpy решение:   [ 1.5 -1.  -1.   1. ]
наше решение:    [1.5, -1.0, -1.0, 1.0]

Макс. расхождение с numpy: 0.0e+00


---

## Конспект: как это работает

### 1. LU-разложение (прямой ход метода Гаусса)

По лекции (слайд 6): последовательно умножаем слева на матрицы $L_k$, зануляя поддиагональные элементы:

$$L_{n-1} \dots L_2 L_1 A = U$$

Тогда:

$$A = L_1^{-1} L_2^{-1} \dots L_{n-1}^{-1} U = LU$$

Множители (слайд 7):

$$l_{jk} = \frac{a_{jk}}{a_{kk}}, \quad k < j \leq n$$

Матрица $L$ (слайд 8):

$$L = L_1^{-1} \dots L_{n-1}^{-1} = \begin{pmatrix} 1 & & & \\ l_{21} & 1 & & \\ \vdots & & \ddots & \\ l_{n1} & \dots & l_{n,n-1} & 1 \end{pmatrix}$$

### 2. Прямая подстановка ($Ly = b$)

Так как $L$ — нижнетреугольная с единицами на диагонали:

$$y_i = b_i - \sum_{j=0}^{i-1} l_{ij} y_j$$

### 3. Обратная подстановка ($Ux = y$)

$$x_i = \frac{y_i - \sum_{j=i+1}^{n-1} u_{ij} x_j}{u_{ii}}$$

### Условие существования (слайд 9)

$A$ имеет LU-разложение $\iff$ $A$ **строго регулярная** (все ведущие подматрицы невырожденные, т.е. $u_{kk} \neq 0$).